The CIFAR-10 dataset that we are using is available directly through the keras module. Therefore, it is easy to download and preprocess. 

In [24]:
# Importing the dataset and other relevant modules
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.constraints import maxnorm
from tensorflow.keras.optimizers import Adam
from keras.layers.convolutional import Conv2D
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers.convolutional import MaxPooling2D
import matplotlib.pyplot as plt
import seaborn as sns

Below, we create our input (X) and output () matrices for training our neural networks. The keras module already provides separate training and test datasets.

In [2]:
# loading the data
(train_X, train_Y), (test_X, test_Y) = cifar10.load_data()

170508288/170498071 [==============================] - 2s 0us/step


Now that we have our data, we need to preprocess it. Specifically, we need to convert the pixel values of the images to floats and then normalize the dataset by dividing by the highest pixel value (255).

In [3]:
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')
 
train_X = train_X/255.0
test_X = test_X/255.0

Finally, we need to convert our output to categorical variables for us to classify. We do this through one-hot encoding. 

In [4]:
train_Y = np_utils.to_categorical(train_Y)
test_Y = np_utils.to_categorical(test_Y)

num_classes=test_Y.shape[1]

We now define our model. 

In [5]:
model=Sequential()
model.add(Conv2D(32,(3,3),input_shape=(32,32,3),padding='same',activation='relu',kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Conv2D(32,(3,3),activation='relu',padding='same',kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(512,activation='relu',kernel_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='relu'))

We create an EarlyStopping function.

In [6]:
es = EarlyStopping(monitor='val_loss', patience=20)

Compiling the model and defining the loss function and the optimizer

In [14]:
model.compile(loss='mean_squared_error', optimizer="adam")

We can now take a look at the summary of our model. 

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 dropout (Dropout)           (None, 32, 32, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 512)               4194816   
                                                        

Finally, we fit the model.

In [15]:
model.fit(train_X, train_Y, validation_data=(test_X, test_Y), epochs=10000000000000, callbacks=[es], batch_size=64, shuffle=True)

Epoch 1/10000000000000
782/782 [==============================] - 13s 15ms/step - loss: 5003.2466 - val_loss: 0.4212
Epoch 2/10000000000000
782/782 [==============================] - 12s 15ms/step - loss: 2.1693 - val_loss: 0.1040
Epoch 3/10000000000000
782/782 [==============================] - 12s 15ms/step - loss: 0.2033 - val_loss: 0.0993
Epoch 4/10000000000000
782/782 [==============================] - 12s 15ms/step - loss: 0.1225 - val_loss: 0.0982
Epoch 5/10000000000000
782/782 [==============================] - 12s 15ms/step - loss: 0.1749 - val_loss: 0.0968
Epoch 6/10000000000000
782/782 [==============================] - 12s 16ms/step - loss: 0.0960 - val_loss: 0.0952
Epoch 7/10000000000000
782/782 [==============================] - 12s 15ms/step - loss: 0.0945 - val_loss: 0.0936
Epoch 8/10000000000000
782/782 [==============================] - 12s 16ms/step - loss: 0.0930 - val_loss: 0.0923
Epoch 9/10000000000000
782/782 [==============================] - 12s 15ms/step - los

Evaluation of our model.

In [18]:
model.evaluate(test_X,test_Y)

313/313 [==============================] - 2s 5ms/step - loss: 0.0910


0.0910063311457634